# Data Scraping
Author: Keanna Knebel

Date: 2020/07/11

In [40]:
# Import packages
import requests
import pandas as pd
import re
import time

from bs4 import BeautifulSoup

In [108]:
# initialize empty dataframe
df = pd.DataFrame(columns =['Season', 'Week', 'Date', 'Favorite', 'Spread', 'Underdog'])

# scrap website for each of the 17 NFL weeks
for week in range(1,18):
    if week == 17:
        page = requests.get(f"http://www.footballlocks.com/nfl_odds_week_{week}.shtml#Closing%20NFL%20Odds%20Week%20{week},%202016-2017")
    else:
        page = requests.get(f"http://www.footballlocks.com/nfl_odds_week_{week}.shtml#Closing%20NFL%20Odds%20Week%20{week},%202016")
    soup = BeautifulSoup(page.text, 'html.parser')
    
    # select weekly data tables from web page
    weekly_tables = soup.find_all('table', attrs={'width':644})
    
    # initialize columns for dataframe
    date = []
    favorite = []
    spread = []
    underdog = []
    df_year = []
    
    for table in weekly_tables:
        
        try:
            year = table.a['name'][-4:]
        except:
            year = year
        
        for rows in table.find_all('tr'):
            cols = rows.find_all('td')

            if len(cols) == 6:
                df_year.append(year)
                date.append(cols[0].get_text())
                favorite.append(cols[1].get_text())
                spread.append(cols[2].get_text())
                underdog.append(cols[3].get_text())

    # create dataframe from the lists of columns
    new_df = pd.DataFrame(list(zip(df_year, date, favorite, spread, underdog)), 
                       columns =['Season', 'Date', 'Favorite', 'Spread', 'Underdog'])
    new_df['Week'] = week
    
    # clean up dataframe by removing blank and title rows
    new_df = new_df.drop_duplicates(subset=['Date', 'Favorite', 'Spread', 'Underdog'], keep=False, ignore_index=True)
    
    # combine dataframes into one
    df = df.append(new_df)
    
    time.sleep(5)

In [109]:
df

,Season,Week,Date,Favorite,Spread,Underdog
0,2019,1,9/5 8:20 ET,At Chicago,-3,Green Bay
1,2019,1,9/8 1:03 ET,At Minnesota,-3.5,Atlanta
2,2019,1,9/8 1:03 ET,At Philadelphia,-10,Washington
3,2019,1,9/8 1:03 ET,At NY Jets,-2.5,Buffalo
4,2019,1,9/8 1:03 ET,Baltimore,-7,At Miami
...,...,...,...,...,...,...
201,2006,17,12/31 4:15 ET,At Philadelphia,-6.5,Atlanta
202,2006,17,12/31 1:00 ET,At Tampa Bay,-3,Seattle
203,2006,17,12/31 1:00 ET,At Tennessee,-3.5,New England
204,2006,17,12/31 4:15 ET,At Denver,-11,San Francisco


In [43]:
# set 'PK' spread to zero 
df.Spread[df.Spread == 'PK'] = 0

In [62]:
# search for postponed or delayed games
df["Postponed"] = [re.search("postponed|delay", x, re.I) for x in df.Date]

In [68]:
df.Postponed[-df.Postponed.isnull()] = 1
df.Postponed[df.Postponed.isnull()] = 0
# iloc 144

In [73]:
df[df.Postponed == 1]

,Year,Week,Date,Favorite,Spread,Underdog,Postponed
39,2017,1,9/10 1:00 ET\nPostponed,Tampa Bay,-2.5,At Miami,1
191,2008,2,Postponedto Week 10,At Houston,-4.5,Baltimore,1
126,2010,5,10/11 8:35 ET\n\nDelay 9:15 ET\n,At NY Jets,-4,Minnesota,1
133,2010,14,Postponed to\nMonday 7:20 ET,NY Giants,-5,At Minnesota,1
131,2010,16,Postponed to\nTuesday 8:00 ET,At Philadelphia,-14,Minnesota,1


In [105]:
#pd.to_numeric(df["Year"])
df.Year.unique()
df2 = df[df.Week == 17]
df2.Year.unique()

array(['2018', '2017', '2016', '2014', '2013', '2012', '1-12', '2011',
       '2010', '2008', '2007', '2006'], dtype=object)

In [36]:
split_date = df.Date.str.split(n = 1, expand = True) 

# making separate first name column from new data frame 
df["Date"]= split_date[0] 
  
# making separate last name column from new data frame 
df["Time"]= split_date[1] 

KeyError: 1

In [39]:
df.Time.unique()

array(['8:20 ET', '1:03 ET', '4:25 ET', '4:05 ET', '7:10 ET', '10:15 ET',
       '1:00 ET', '10:20 ET', '8:30 ET', '1:00 ET\nPostponed', '8:35 ET',
       '10:25 ET', '8:25 ET', '7:00 ET', '4:15 ET', '8:15 ET', '8:40 ET',
       'Week 10', '9:30 ET', '9:30 AM', '1;00 ET', '11:35 ET',
       '8:35 ET\n\nDelay 9:15 ET\n', '12:35 ET', '4:30 ET', '12:30 ET',
       '1:00 ET\nPPD to MNF', '8:00 ET', 'to\nMonday 7:20 ET', '7:20 ET',
       '7:30 ET', 'to\nTuesday 8:00 ET', '5:00 ET'], dtype=object)

In [ ]:
## TO DO ##

# clean date column
# remove 'at' from team names